In [0]:
%sql
SELECT 'device_messages_raw' AS table_name, COUNT(*) AS rows
FROM workspace.bronze.device_messages_raw
UNION ALL
SELECT 'rapid_step_tests_raw', COUNT(*)
FROM workspace.bronze.rapid_step_tests_raw;


In [0]:
%sql
SELECT device_Id, COUNT(*) AS messages
FROM workspace.bronze.device_messages_raw
GROUP BY device_Id
ORDER BY messages DESC
LIMIT 10;


We standardize distance into numeric centimeters for a better evaluation

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW device_messages_clean AS
SELECT
  device_Id,
  TRY_CAST(
    REGEXP_REPLACE(CAST(distance AS STRING), '[^0-9.]', '')
    AS DOUBLE
  ) AS distance_cm,
  CAST(timestamp AS BIGINT) AS ts_ms,
  CAST(date AS BIGINT) AS date_ms,
  message_Origin,
  sensor_Type,
  message
FROM workspace.bronze.device_messages_raw;


This temporary view cleans and standardizes distance values from raw device messages. Because TEMP VIEWs do not persist across sessions, this cell must be run before downstream queries.

In [0]:
%sql
SELECT device_Id,
       AVG(distance_cm) AS avg_cm,
       MIN(distance_cm) AS min_cm,
       MAX(distance_cm) AS max_cm
FROM device_messages_clean
GROUP BY device_Id
ORDER BY avg_cm DESC
LIMIT 10;

**Per-device distance statistics**

This query calculates the average, minimum, and maximum cleaned distance values (in centimeters) for each device. These summary statistics help validate that the distance field was cleaned correctly and provide an overview of sensor behavior across devices before joining with step test dat

In [0]:
%sql
SELECT
  device_Id,
  COUNT(*) AS test_count,
  AVG(stop_Time - start_Time) AS avg_duration_ms,
  MIN(stop_Time - start_Time) AS min_duration_ms,
  MAX(stop_Time - start_Time) AS max_duration_ms,
  AVG(total_Steps) AS avg_total_steps,
  MIN(total_Steps) AS min_total_steps,
  MAX(total_Steps) AS max_total_steps
FROM workspace.bronze.rapid_step_tests_raw
GROUP BY device_Id
ORDER BY test_count DESC
LIMIT 20;


**SQL 1.5 — Rapid Step Test Summaries**

This query evaluates rapid step test data by device. It counts the number of tests per device and calculates summary statistics for test duration and total step counts. These summaries help validate the quality and consistency of the step test data before performing joins with device message readings.

In [0]:
%sql
WITH exploded_steps AS (
  SELECT
    customer,
    device_Id,
    start_Time,
    stop_Time,
    test_Time,
    total_Steps,
    posexplode(step_Points) AS (step_index, step_ms)
  FROM workspace.bronze.rapid_step_tests_raw
)

SELECT
  customer,
  device_Id,
  start_Time,
  stop_Time,
  COUNT(*) AS steps_observed,
  AVG(step_ms) AS avg_step_ms,
  MIN(step_ms) AS min_step_ms,
  MAX(step_ms) AS max_step_ms,
  STDDEV(step_ms) AS sd_step_ms
FROM exploded_steps
GROUP BY customer, device_Id, start_Time, stop_Time
ORDER BY steps_observed DESC
LIMIT 20;


In [0]:
%sql
WITH tests AS (
  SELECT
    customer,
    device_Id,
    start_Time,
    stop_Time,
    test_Time,
    total_Steps
  FROM workspace.bronze.rapid_step_tests_raw
),
msgs AS (
  SELECT
    device_Id,
    ts_ms,
    distance_cm
  FROM device_messages_clean
  WHERE distance_cm IS NOT NULL
)

SELECT
  t.customer,
  t.device_Id,
  t.start_Time,
  t.stop_Time,
  t.test_Time,
  t.total_Steps,

  COUNT(m.ts_ms)     AS readings_in_window,
  AVG(m.distance_cm) AS avg_cm_in_window,
  MIN(m.distance_cm) AS min_cm_in_window,
  MAX(m.distance_cm) AS max_cm_in_window,
  STDDEV(m.distance_cm) AS sd_cm_in_window

FROM tests t
JOIN msgs m
  ON m.device_Id = t.device_Id
 AND m.ts_ms BETWEEN t.start_Time AND t.stop_Time

GROUP BY
  t.customer, t.device_Id, t.start_Time, t.stop_Time, t.test_Time, t.total_Steps

ORDER BY readings_in_window DESC
LIMIT 20;


**SQL 1.7 — Time-window join**

This query joins rapid step tests with device message readings using both device_id and the test time window. For each test, it aggregates the number of sensor readings and summarizes distance values observed during the test period. These window-based aggregates serve as feature data for validating sensor behavior during step tests.

**Python 2.1 — Load Bronze Tables**

In this step, we load the two raw STEDI datasets from the bronze layer (workspace.bronze) into Spark DataFrames. We then preview a few rows and confirm the schemas so later cleaning and joins match the expected column names and data type

In [0]:
# Python 2.1 — Load Bronze Tables with Spark

device_messages_df = spark.table("workspace.bronze.device_messages_raw")
rapid_step_tests_df = spark.table("workspace.bronze.rapid_step_tests_raw")

print("Device Messages Schema:")
device_messages_df.printSchema()

print("\nRapid Step Tests Schema:")
rapid_step_tests_df.printSchema()

display(device_messages_df.limit(10))
display(rapid_step_tests_df.limit(10))


**Python 2.2 — Clean distance and standardize timestamps**

This step cleans the raw distance field by removing non-numeric characters and casting it to a numeric value (distance_cm). It also standardizes timestamp columns by casting them to BIGINT so they can be reliably used for filtering and time-window joins with step test data.

In [0]:
from pyspark.sql import functions as F

# Clean and standardize the device messages data
device_messages_clean_df = (
    device_messages_df
    .withColumn("distance_str", F.col("distance").cast("string"))
    .withColumn("distance_cm", F.regexp_replace("distance_str", r"[^0-9.]", "").cast("double"))
    .withColumn("ts_ms", F.col("timestamp").cast("bigint"))
    .withColumn("date_ms", F.col("date").cast("bigint"))
)

# Quick validation checks
display(device_messages_clean_df.select("device_Id", "distance", "distance_cm", "timestamp", "ts_ms").limit(20))

# Optional: filter out null/blank cleaned values for later steps
device_messages_clean_nonnull_df = device_messages_clean_df.filter(F.col("distance_cm").isNotNull())

print("Rows (raw):", device_messages_df.count())
print("Rows (clean non-null distance):", device_messages_clean_nonnull_df.count())


**Python 2.3 — Per-device distance statistics**

This step groups the cleaned device message data by deviceId and calculates descriptive statistics on distance_cm (count, average, minimum, maximum). These stats confirm the cleaning worked and provide baseline device behavior before we join device messages with step test windows.

In [0]:
from pyspark.sql import functions as F

# Per-device stats using cleaned, non-null distance values
device_stats_df = (
    device_messages_clean_nonnull_df
    .groupBy("device_Id")
    .agg(
        F.count("*").alias("message_count"),
        F.avg("distance_cm").alias("avg_cm"),
        F.min("distance_cm").alias("min_cm"),
        F.max("distance_cm").alias("max_cm")
    )
    .orderBy(F.desc("message_count"))
)

display(device_stats_df.limit(20))


**Python 2.4 — Explode stepPoints and compute step-level statistics**

Each rapid step test contains an array field (stepPoints) with step timing values. This step explodes the array into one row per step so we can compute step-level statistics (count, average, min/max, standard deviation) for each test window per device.

In [0]:
from pyspark.sql import functions as F

# Explode stepPoints into one row per step
steps_exploded_df = (
    rapid_step_tests_df
    .select(
        "customer",
        "device_Id",
        "start_Time",
        "stop_Time",
        "test_Time",
        "total_Steps",
        F.posexplode("step_Points").alias("step_index", "step_ms")
    )
)

display(steps_exploded_df.limit(20))

# Compute step-level stats per test (grouped by test window)
step_stats_df = (
    steps_exploded_df
    .groupBy("customer", "device_Id", "start_Time", "stop_Time")
    .agg(
        F.count("*").alias("steps_observed"),
        F.avg("step_ms").alias("avg_step_ms"),
        F.min("step_ms").alias("min_step_ms"),
        F.max("step_ms").alias("max_step_ms"),
        F.stddev("step_ms").alias("sd_step_ms")
    )
    .orderBy(F.desc("steps_observed"))
)

display(step_stats_df.limit(20))


**Python 2.5 — Time-window join and feature aggregates**

This step joins rapid step tests to device message readings using deviceId and the test time window (startTime to stopTime). After joining, it aggregates the sensor readings observed during each test window (count, average, min, max, and standard deviation of distance_cm). These aggregated values act as feature data for validating sensor behavior during step tests and for downstream analytics.

In [0]:
from pyspark.sql import functions as F

# Select only the columns we need
tests_df = rapid_step_tests_df.select(
    "customer", "device_Id", "start_Time", "stop_Time", "test_Time", "total_Steps"
)

msgs_df = device_messages_clean_nonnull_df.select(
    "device_Id", "ts_ms", "distance_cm"
)

# Time-window join: messages during the test window
joined_df = (
    tests_df.alias("t")
    .join(
        msgs_df.alias("m"),
        (F.col("t.device_Id") == F.col("m.device_Id")) &
        (F.col("m.ts_ms").between(F.col("t.start_Time"), F.col("t.stop_Time"))),
        how="inner"
    )
)

# Feature aggregates per test window
features_df = (
    joined_df
    .groupBy(
        "t.customer", "t.device_Id", "t.start_Time", "t.stop_Time", "t.test_Time", "t.total_Steps"
    )
    .agg(
        F.count("m.ts_ms").alias("readings_in_window"),
        F.avg("m.distance_cm").alias("avg_cm_in_window"),
        F.min("m.distance_cm").alias("min_cm_in_window"),
        F.max("m.distance_cm").alias("max_cm_in_window"),
        F.stddev("m.distance_cm").alias("sd_cm_in_window")
    )
    .orderBy(F.desc("readings_in_window"))
)

display(features_df.limit(20))
print("Feature rows:", features_df.count())


**Python 2.6 — Visual data check (distance over time for one device)**

This plot provides a quick visual validation of the cleaned sensor readings by graphing distance_cm over time (ts_ms) for a single device. A visual check helps confirm the cleaning worked and that the distance values change in a realistic pattern.

In [0]:
import matplotlib.pyplot as plt
from pyspark.sql import functions as F

# Pick a deviceId from the feature table (ensures it has matching test+message data)
sample_device = features_df.select("device_Id").first()["device_Id"]
print("Sample deviceId:", sample_device)

# Pull a manageable number of rows for plotting
plot_pdf = (
    device_messages_clean_nonnull_df
    .filter(F.col("device_Id") == sample_device)
    .select("ts_ms", "distance_cm")
    .orderBy("ts_ms")
    .limit(1000)
    .toPandas()
)

plt.figure()
plt.plot(plot_pdf["ts_ms"], plot_pdf["distance_cm"])
plt.title(f"Distance over time for device {sample_device}")
plt.xlabel("Timestamp (ms)")
plt.ylabel("Distance (cm)")
plt.show()


Cleaning and joining the datasets was straightforward once the timestamp formats were aligned, and the time-window joins worked as expected. The most confusing part was ensuring that all timestamps used the same unit (milliseconds) to avoid empty joins. One ethics risk is that incorrect data cleaning or timestamp handling could introduce misleading patterns, which could affect decisions or models built on this data.